# How to use llm judge template?

In [ ]:
from evidently.descriptors import LLMEval, NegativityLLMEval, PIILLMEval, DeclineLLMEval

In [ ]:
import pandas as pd
import numpy as np

from datetime import datetime
from datetime import time
from datetime import timedelta

import requests
from io import BytesIO

from sklearn import datasets, ensemble, model_selection

In [ ]:
from evidently.ui.workspace.cloud import CloudWorkspace

from evidently import ColumnMapping
from evidently.report import Report

from evidently.metrics import ColumnSummaryMetric

from evidently.metric_preset import DataQualityPreset, TextOverviewPreset, TextEvals

## Load Data

In [ ]:
response = requests.get("https://raw.githubusercontent.com/evidentlyai/evidently/main/examples/how_to_questions/chat_df.csv")
csv_content = BytesIO(response.content)

In [ ]:
assistant_logs = pd.read_csv(csv_content, index_col=0, parse_dates=['start_time', 'end_time'])
assistant_logs.index = assistant_logs.start_time
assistant_logs.index.rename('index', inplace=True)

In [ ]:
pd.set_option('display.max_colwidth', None)

In [ ]:
assistant_logs[["question", "response"]].head()

## One-off reports

In [ ]:
column_mapping = ColumnMapping(
    datetime='start_time',
    datetime_features=['end_time'],
    text_features=['question', 'response'],
    categorical_features=['organization', 'model_ID', 'region', 'environment', 'feedback'],
)

### LLM-based descriptors without parameters

In [ ]:
report = Report(metrics=[
    TextEvals(column_name="question"),
    TextEvals(column_name="response")
])

report.run(reference_data=assistant_logs[datetime(2024, 4, 8) : datetime(2024, 4, 9)][:10], 
           current_data=assistant_logs[datetime(2024, 4, 9) : datetime(2024, 4, 10)][:10], 
           column_mapping=column_mapping)
report 

### LLM-based descriptors without parameters

In [ ]:
report = Report(metrics=[
    TextEvals(column_name="question", descriptors=[
        NegativityLLMEval()   
    ]),
    TextEvals(column_name="response", descriptors=[
        PIILLMEval(), 
        DeclineLLMEval()
    ])
])

report.run(reference_data=assistant_logs[datetime(2024, 4, 8) : datetime(2024, 4, 9)][:10], 
           current_data=assistant_logs[datetime(2024, 4, 9) : datetime(2024, 4, 10)][:10], 
           column_mapping=column_mapping)
report 

### LLM-based descriptors with parameters

In [ ]:
report = Report(metrics=[
    TextEvals(column_name="question", descriptors=[
        NegativityLLMEval(include_category=True)   
    ]),
    TextEvals(column_name="response", descriptors=[
        PIILLMEval(include_reasoning=False), 
        DeclineLLMEval(include_score=True)
    ])
])

report.run(reference_data=assistant_logs[datetime(2024, 4, 8) : datetime(2024, 4, 9)][:10], 
           current_data=assistant_logs[datetime(2024, 4, 9) : datetime(2024, 4, 10)][:10], 
           column_mapping=column_mapping)
report 

### Custom LLM-based descriptor

In [ ]:
from evidently.features.llm_judge import BinaryClassificationPromptTemplate

In [ ]:
custom_judge = LLMEval(
    subcolumn="category",
    template = BinaryClassificationPromptTemplate(      
        criteria = """Conciseness refers to the quality of being brief and to the point, while still providing all necessary information.
            A concise response should:
            - Provide the necessary information without unnecessary details or repetition.
            - Be brief yet comprehensive enough to address the query.
            - Use simple and direct language to convey the message effectively.
        """,
        target_category="concise",
        non_target_category="verbose",
        uncertainty="unknown",
        include_reasoning=True,
        pre_messages=[("system", "You are a judge which evaluates text.")],
        ),
    provider = "openai",
    model = "gpt-4o-mini"
)

report = Report(metrics=[
    TextEvals(column_name="response", descriptors=[
        custom_judge(display_name="test")
    ])
])

report.run(reference_data=assistant_logs[datetime(2024, 4, 8) : datetime(2024, 4, 9)][:10], 
           current_data=assistant_logs[datetime(2024, 4, 9) : datetime(2024, 4, 10)][:10], 
           column_mapping=column_mapping)
report 

In [ ]:
custom_judge = LLMEval(
    subcolumn="score",
    template = BinaryClassificationPromptTemplate(      
        criteria = """Conciseness refers to the quality of being brief and to the point, while still providing all necessary information.
            A concise response should:
            - Provide the necessary information without unnecessary details or repetition.
            - Be brief yet comprehensive enough to address the query.
            - Use simple and direct language to convey the message effectively.
        """,
       target_category="concise",
        non_target_category="verbose",
        uncertainty="unknown",
        include_reasoning=True,
        include_score=True,
        pre_messages=[("system", "You are a judge which evaluates text.")],
        ),
    provider = "openai",
    model = "gpt-4o-mini"
)

report = Report(metrics=[
    TextEvals(column_name="response", descriptors=[
        custom_judge
    ])
])

report.run(reference_data=assistant_logs[datetime(2024, 4, 8) : datetime(2024, 4, 9)][:10], 
           current_data=assistant_logs[datetime(2024, 4, 9) : datetime(2024, 4, 10)][:10], 
           column_mapping=column_mapping)
report 

In [ ]:
multi_column_judge = LLMEval(
        subcolumn="category",
        additional_columns={"question": "question"},
        template=BinaryClassificationPromptTemplate(
            criteria=""""Relevance" refers to the response directly addresses the question and effectively meets the user's intent.  
Relevant answer is an answer that directly addresses the question and effectively meets the user's intent.

=====
{question}
=====
            """,
            target_category="Relevant",
            non_target_category="Irrelevant",
            include_reasoning=True,
            pre_messages=[("system",
                           "You are an expert evaluator assessing the quality of a Q&A system. Your goal is to determine if the provided answer is relevant to the question based on the criteria below.")],
        ),
        provider="openai",
        model="gpt-4o-mini",
        display_name="Relevancy"
    )

report = Report(metrics=[
    TextEvals(column_name="response", descriptors=[
        multi_column_judge
    ])
])

report.run(reference_data=assistant_logs[datetime(2024, 4, 8) : datetime(2024, 4, 9)][:10], 
           current_data=assistant_logs[datetime(2024, 4, 9) : datetime(2024, 4, 10)][:10], 
           column_mapping=column_mapping)
report 